In [ ]:
from fins3666.defines import *
from fins3666.utils import load_dataframe, getExchangeRate, currencyUSDvals
from fins3666.portfolio import Portfolio, AssetDict


fx = load_dataframe(os.path.join(DIR_DATA_RAW, 'fx-historical.csv'))
ir = load_dataframe(os.path.join(DIR_DATA_RAW, 'ir-historical.csv'))

# determine correlation between columns of fx
correlation = fx.corr()
# print(correlation)

currs = ['AUD', 'CAD', 'CHF']
time = pd.to_datetime('2020-01')


SamplePortfolio = Portfolio(
    assets=currs, sizes=[100, -200, 273], unit_values=currencyUSDvals(fx, timestamp=time, currencies=currs), timestamp=time)

# dict containing -> {name:'str', units:float64, purchase_price:float64, timestamp:datetime[64]}
assets = [{}, ]
SamplePortfolio.AquireAssets()

print(str(SamplePortfolio))



                               Portfolio Overview                               
Asset         Units        Unit Value    Asset Value       Position    Weight
AUD           100.00          $0.67         $66.94           Long    33.68%
CAD          -200.00          $0.76       $-151.22          Short   -76.07%
CHF           273.00          $1.04        $283.06           Long   142.40%

Portfolio - 3 assets
Unit Price Timestamp: 2020-01-01 00:00:00,
Net Value ($USD): $198.78
